<a href="https://colab.research.google.com/github/farshidhesami/iNeuron_Assignment/blob/main/Logistic_Regression_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install statsmodels scikit-learn matplotlib patsy

In [ ]:
!pip install statsmodels


### Loading and Preparing the Data


In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from patsy import dmatrices
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import metrics

# Load the dataset
dta = sm.datasets.fair.load_pandas().data

# Add 'affair' column: 1 represents having affairs, 0 represents not
dta['affair'] = (dta.affairs > 0).astype(int)

y, X = dmatrices('affair ~ rate_marriage + age + yrs_married + children + religious + educ + C(occupation) + C(occupation_husb)', dta, return_type="dataframe")
X = X.rename(columns = {
    'C(occupation)[T.2.0]':'occ_2',
    'C(occupation)[T.3.0]':'occ_3',
    'C(occupation)[T.4.0]':'occ_4',
    'C(occupation)[T.5.0]':'occ_5',
    'C(occupation)[T.6.0]':'occ_6',
    'C(occupation_husb)[T.2.0]':'occ_husb_2',
    'C(occupation_husb)[T.3.0]':'occ_husb_3',
    'C(occupation_husb)[T.4.0]':'occ_husb_4',
    'C(occupation_husb)[T.5.0]':'occ_husb_5',
    'C(occupation_husb)[T.6.0]':'occ_husb_6'
})

y = np.ravel(y)


### Splitting the Data

In [ ]:
from sklearn.model_selection import train_test_split

# Splitting data into 70% training and 30% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


### Training the Logistic Regression Model

In [ ]:
from sklearn.linear_model import LogisticRegression

# Initialize the model
model = LogisticRegression(max_iter=10000)

# Fit the model to the training data
model.fit(X_train, y_train)


LogisticRegression(max_iter=10000)

### Evaluating the Model

In [ ]:
from sklearn import metrics

# Making predictions on the testing set
y_pred = model.predict(X_test)

# Calculating the accuracy of the model
accuracy = metrics.accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# Displaying the confusion matrix
conf_matrix = metrics.confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(conf_matrix)

# Displaying the classification report
class_report = metrics.classification_report(y_test, y_pred)
print('Classification Report:')
print(class_report)


Accuracy: 0.724607329842932
Confusion Matrix:
[[1140  135]
 [ 391  244]]
Classification Report:
              precision    recall  f1-score   support

         0.0       0.74      0.89      0.81      1275
         1.0       0.64      0.38      0.48       635

    accuracy                           0.72      1910
   macro avg       0.69      0.64      0.65      1910
weighted avg       0.71      0.72      0.70      1910



### Deploying  Model

In [ ]:
import joblib

# Save the model to a file
joblib.dump(model, 'model.pkl')

# If you're in a Colab environment, make sure to download it to your local machine
from google.colab import files
files.download('model.pkl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Flask Application :
 - We use flask-ngrok because Colab notebooks are run on a remote server, and ngrok is a tool that can tunnel traffic from a public URL to your local Flask app running in the notebook.


In [ ]:
!pip install flask-ngrok


In [9]:
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
import joblib
import pandas as pd  # Make sure to import pandas if you're using it

# If your model is not uploaded to Colab, make sure to upload it now
# You can use the Colab file upload feature (in the left sidebar under 'Files')

app = Flask(__name__)
run_with_ngrok(app)  # This starts ngrok when the app is run

@app.route("/")
def home():
    return "Welcome to the Flask app running in Colab!"

@app.route('/predict', methods=['POST'])
def predict():
    json_ = request.json
    query_df = pd.DataFrame(json_)
    # Ensure the model 'model.pkl' has been uploaded or is accessible in your environment
    model = joblib.load('model.pkl')  # Load the model at the point of making a prediction
    prediction = model.predict(query_df)
    return jsonify({'prediction': list(prediction)})

if __name__ == "__main__":
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-10:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connection.py", line 203, in _new_conn
    sock = connection.create_connection(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 791, in urlopen
    response = self._make_request(
  File "/usr/local/lib/python3.10/dist-packages/urll